In [1]:
!pip install --proxy 10.0.0.1:3128 setuptools --upgrade

Requirement already up-to-date: setuptools in c:\users\admin\appdata\local\programs\python\python38\lib\site-packages (67.7.2)


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [44]:
!pip install --proxy 10.0.0.1:3128 dostoevsky gensim pandas emoji sklearn scikit-learn

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


## Импорт модели Word2Vec

In [43]:
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pandas as pd
from dostoevsky.models import FastTextSocialNetworkModel
from dostoevsky.tokenization import RegexTokenizer
import emoji
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB

ModuleNotFoundError: No module named 'sklearn'

In [4]:
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [5]:
wv.wv.most_similar('😠')

[('виктория', 0.9023991227149963),
 ('бегать', 0.9014540314674377),
 ('😰', 0.9013167023658752),
 ('нынешний', 0.901007354259491),
 ('вооружение', 0.900209903717041),
 ('медицина', 0.8998721837997437),
 ('приходить', 0.8997824192047119),
 ('приводить', 0.8997141122817993),
 ('плевать', 0.8997119665145874),
 ('тихий', 0.89943528175354)]

## Загрузка датасета

In [6]:
colnames = ['comment',]

In [7]:
comments = pd.read_csv('datasets/vk.csv', names=colnames, header=0, sep='\t')

In [8]:
comments

,comment
0,['😭']
1,"['хранить', 'господь', 'неравнодушный', 'челов..."
2,"['евгений', 'еслиб', 'бог', 'такой', 'допускать']"
3,"['кирилл', 'полностью', 'согласный', '😢']"
4,"['кирилл', 'бог', 'путь', 'неисповедимый']"
...,...
50779,"['с', 'фамилия', '🙄']"
50780,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
50781,"['чудо', 'чудной', 'диво', 'дивный']"
50782,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [9]:
def str_to_list(i):
    new_list = eval(i)
    return new_list

In [10]:
comments['comment'] = comments['comment'].apply(str_to_list)

In [11]:
def list_to_str(doc):
    string = ' '.join(doc)
    return string

In [12]:
comments['comment'] = comments['comment'].apply(list_to_str)

In [13]:
comments

,comment
0,😭
1,хранить господь неравнодушный человек 🙏 🕊
2,евгений еслиб бог такой допускать
3,кирилл полностью согласный 😢
4,кирилл бог путь неисповедимый
...,...
50779,с фамилия 🙄
50780,наш день повезти мужик фамилия но суть это мен...
50781,чудо чудной диво дивный
50782,пригреться уснуть свариться лягушка так народ ...


## Применение модели Dostoevsky

In [14]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

#### Проверяем tokenizer

In [15]:
tokens = tokenizer.split(comments.iloc[1]['comment'])

In [16]:
tokens

[('хранить', None),
 ('господь', None),
 ('неравнодушный', None),
 ('человек', None),
 ('🙏', None),
 ('🕊', None)]

#### Выборка содержащих эмодзи сообщений

In [17]:
comments_emoji = pd.DataFrame()

In [18]:
def get_emoji(doc):
    flag = 0
    for c in doc.split():
      if emoji.is_emoji(c):
        flag+=1
    if flag==0:
        doc = np.nan
    return doc

In [19]:
comments_emoji['comment'] = comments['comment'].apply(get_emoji)

In [20]:
comments_emoji = comments_emoji.dropna()

In [21]:
comments_emoji = comments_emoji.reset_index(drop=True)

In [22]:
comments_emoji

,comment
0,😭
1,хранить господь неравнодушный человек 🙏 🕊
2,кирилл полностью согласный 😢
3,каво 🚬 🗿
4,ой ой ой смотреть какой обиженный боз обизятьс...
...,...
8903,сентябрь 😭
8904,ер вылезать дворец культура город страница пре...
8905,репост приветствоваться 😉
8906,платить сгк новый ахта 🤣 старый арестовать ита...


In [23]:
comments_list = comments_emoji['comment'].values.tolist()

## Создание обучающей выборки

In [24]:
negative = ['😡', '👿', '😾', '😤', '😠', '🤬', '💀', '☠️', '😭', '🖕', '🚫', '👎', '👹', '😿', '👺', '😨']

In [25]:
sarcastic = ['🤡', '🙃', '🤢', '😈', '😐', '🤣', '😅', '🤦', '😆', '💩', '🤔', '🐓', '🐷', '🐽', '🐖', '🌈', '🤮', '🤷', '😕']

In [26]:
positive = ['😀', '😃', '😉', '👍', '😎', '😁', '🙂', '🤗', '👏', '😸', '🙏', '😏', '💪', '🙌', '🙏', '🔥'] 

In [27]:
results = model.predict(comments_list, k = 2)

In [32]:
for message, sentiment in zip(comments_list, results):
    print(message, '----->;', sentiment)

😭 ----->; {'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
хранить господь неравнодушный человек 🙏 🕊 ----->; {'neutral': 0.5156299471855164, 'negative': 0.3276783227920532}
кирилл полностью согласный 😢 ----->; {'neutral': 0.7310686111450195, 'positive': 0.061885979026556015}
каво 🚬 🗿 ----->; {'neutral': 0.3557848632335663, 'skip': 0.13661839067935944}
ой ой ой смотреть какой обиженный боз обизяться ти 🥺 ----->; {'skip': 0.523430347442627, 'negative': 0.348655104637146}
👍 😘 ----->; {'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
наиля красавец вырасти 👍 👍 👍 👍 ----->; {'positive': 0.8903042674064636, 'skip': 0.016924913972616196}
аватарка игра великий султан 🤭 ----->; {'neutral': 0.8962613940238953, 'skip': 0.11280541121959686}
жена сразу развестись подарить такой йухня 😁 ----->; {'neutral': 0.320831298828125, 'positive': 0.23935934901237488}
эх первый фотка идеальный карта пяточек оторвать 🤣 ----->; {'positive': 0.546748161315918, 'neutral': 0.2337

In [28]:
tonality_itog = list()
for comment, sentiment in zip(comments_list, results):
    tonality = list()
    tonality.append(list(sentiment.keys())[0])
    tonality.append(list(sentiment.keys())[1])
    for c in comment.split():
        if emoji.is_emoji(c):
            if c in negative:
                tonality.append('negative')
            elif c in positive:
                tonality.append('positive')
            elif c in sarcastic:
                tonality.append('sarcastic')
    most_com = Counter(tonality).most_common(1)
    tonality_itog.append(list(dict(most_com).keys())[0])

In [30]:
comments_emoji['tonality'] = tonality_itog

In [31]:
comments_emoji

,comment,tonality
0,😭,negative
1,хранить господь неравнодушный человек 🙏 🕊,neutral
2,кирилл полностью согласный 😢,neutral
3,каво 🚬 🗿,neutral
4,ой ой ой смотреть какой обиженный боз обизятьс...,skip
...,...,...
8903,сентябрь 😭,neutral
8904,ер вылезать дворец культура город страница пре...,sarcastic
8905,репост приветствоваться 😉,neutral
8906,платить сгк новый ахта 🤣 старый арестовать ита...,neutral


In [35]:
vectors_all = list()
for doc in comments_emoji['comment']:
    vectors = list()
    for word in doc.split():
        try:
            vectors.append(wv.wv[word])
        except KeyError:
            print('Слова ' + word + ' нет в словаре word2vec модели!')
    if vectors == []:
        vectors = np.nan
    vectors_all.append(vectors)

Слова 🕊 нет в словаре word2vec модели!
Слова боз нет в словаре word2vec модели!
Слова обизяться нет в словаре word2vec модели!
Слова йухня нет в словаре word2vec модели!
Слова мелодрама нет в словаре word2vec модели!
Слова кличка нет в словаре word2vec модели!
Слова гуталин нет в словаре word2vec модели!
Слова госзаказ нет в словаре word2vec модели!
Слова ринуться нет в словаре word2vec модели!
Слова загибаться нет в словаре word2vec модели!
Слова фб нет в словаре word2vec модели!
Слова блогерка нет в словаре word2vec модели!
Слова неодушевлённый нет в словаре word2vec модели!
Слова пованивать нет в словаре word2vec модели!
Слова театральщина нет в словаре word2vec модели!
Слова авиастроение нет в словаре word2vec модели!
Слова судостроение нет в словаре word2vec модели!
Слова приборостроение нет в словаре word2vec модели!
Слова судостроительство нет в словаре word2vec модели!
Слова «император нет в словаре word2vec модели!
Слова «великое нет в словаре word2vec модели!
Слова луки» нет 

In [36]:
comments_emoji['vectors'] = vectors_all

In [37]:
comments_emoji

,comment,tonality,vectors
0,😭,negative,"[[-0.0222436, 0.08431921, 0.028719699, 0.01434..."
1,хранить господь неравнодушный человек 🙏 🕊,neutral,"[[-0.0042343023, 0.032415874, 0.01050371, 0.00..."
2,кирилл полностью согласный 😢,neutral,"[[-0.0034460896, 0.012640278, 0.0039798515, 0...."
3,каво 🚬 🗿,neutral,"[[-0.0033172567, 0.0026729992, 0.0015118181, -..."
4,ой ой ой смотреть какой обиженный боз обизятьс...,skip,"[[-0.014190151, 0.05540202, 0.02489116, 0.0105..."
...,...,...,...
8903,сентябрь 😭,neutral,"[[-0.002400653, 0.010368688, 0.0060007605, -0...."
8904,ер вылезать дворец культура город страница пре...,sarcastic,"[[0.0022103162, 0.00069588295, 6.6017776e-05, ..."
8905,репост приветствоваться 😉,neutral,"[[0.0013026254, 0.002365306, 0.001832386, -0.0..."
8906,платить сгк новый ахта 🤣 старый арестовать ита...,neutral,"[[-0.018510455, 0.07289963, 0.030422136, 0.016..."


In [38]:
comments_emoji['vectors'] = comments_emoji['vectors'].dropna()

In [40]:
comments_emoji = comments_emoji.reset_index(drop=True)

In [41]:
comments_emoji

,comment,tonality,vectors
0,😭,negative,"[[-0.0222436, 0.08431921, 0.028719699, 0.01434..."
1,хранить господь неравнодушный человек 🙏 🕊,neutral,"[[-0.0042343023, 0.032415874, 0.01050371, 0.00..."
2,кирилл полностью согласный 😢,neutral,"[[-0.0034460896, 0.012640278, 0.0039798515, 0...."
3,каво 🚬 🗿,neutral,"[[-0.0033172567, 0.0026729992, 0.0015118181, -..."
4,ой ой ой смотреть какой обиженный боз обизятьс...,skip,"[[-0.014190151, 0.05540202, 0.02489116, 0.0105..."
...,...,...,...
8903,сентябрь 😭,neutral,"[[-0.002400653, 0.010368688, 0.0060007605, -0...."
8904,ер вылезать дворец культура город страница пре...,sarcastic,"[[0.0022103162, 0.00069588295, 6.6017776e-05, ..."
8905,репост приветствоваться 😉,neutral,"[[0.0013026254, 0.002365306, 0.001832386, -0.0..."
8906,платить сгк новый ахта 🤣 старый арестовать ита...,neutral,"[[-0.018510455, 0.07289963, 0.030422136, 0.016..."


## Обучение полиномиального наивного байесовского классификатора